In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [4]:
dad = pd.read_parquet("./data/DAD21.parquet")
dt = pd.read_parquet("./data/DT21.parquet")
pop = pd.read_csv("./data/pop21.csv", delimiter=";")
acti = pd.read_csv('./data/acti.csv', delimiter=";", dtype={'CODGEO': 'str'})

In [5]:
dad.columns =['pop21','code_a','code']
dt.columns = ['pop21','code','code_a']

dad_edges=dad.groupby(['code','code_a']).pop21.sum()
dad_edges = dad_edges.reset_index()

dt_edges=dt.groupby(['code','code_a']).pop21.sum()
dt_edges = dt_edges.reset_index()

dad_edges_final = dad_edges.loc[dad_edges['pop21']>50].copy()
dt_edges_final = dt_edges.loc[dt_edges['pop21']>50].copy()

pop = pop[['COM','PMUN']]
pop.columns = ['code','pop']
dad_edges_final = dad_edges_final.merge(pop, how='left', on='code')
dad_edges_final['migra']=dad_edges_final.apply(lambda x: float(x['pop21'])/x['pop'], axis=1)
dad_edges_final = dad_edges_final[['code','code_a','migra']]

acti = acti[['CODGEO', 'P21_POP1564']]
acti.columns = ['code','pop']
dt_edges_final = dt_edges_final.merge(acti, how='left', on='code')
dt_edges_final['d_t']=dt_edges_final.apply(lambda x: float(x['pop21'])/x['pop'], axis=1)
dt_edges_final = dt_edges_final[['code','code_a','d_t']]

dt_edges_final = dt_edges_final.loc[~dt_edges_final['code_a'].isin(['YYYYY', '99999'])].copy()
dad_edges_final = dad_edges_final.loc[~dad_edges_final['code_a'].isin(['YYYYY', '99999'])].copy()

In [6]:
edges_final = dt_edges_final.merge(dad_edges_final, how='outer', on=['code','code_a']).fillna(0)

In [7]:
edges_final.to_parquet('./data_GNN/edges.parquet')

In [13]:
routes = gpd.read_file("./data/bdtopo_routes.gpkg", columns = ['ID','NATURE','IMPORTANCE','ETAT','FICTIF','geometry'])

In [14]:
routes = routes.loc[routes['FICTIF']=='Non' ].copy()

,ID,NATURE,IMPORTANCE,FICTIF,ETAT,geometry
0,TRONROUT0000000009049190,Route à 1 chaussée,5,Non,En service,"LINESTRING (837481.1 6557857.9, 837491.8 65578..."
1,TRONROUT0000000076837818,Route à 1 chaussée,5,Non,En service,"LINESTRING (837629.2 6558024.9, 837623.9 65580..."
2,TRONROUT0000000008993435,Route à 1 chaussée,5,Non,En service,"LINESTRING (867273.6 6576237.7, 867272.8 65762..."
3,TRONROUT0000002218413997,Route à 1 chaussée,5,Non,En service,"LINESTRING (867207.8 6576232.9, 867242.9 65762..."
4,TRONROUT0000002218413998,Route à 1 chaussée,5,Non,En service,"LINESTRING (867337.9 6576285.2, 867333 6576272..."
...,...,...,...,...,...,...
9995,TRONROUT0000000009046898,Route à 1 chaussée,5,Non,En service,"LINESTRING (870050.4 6558999.5, 870055.4 65589..."
9996,TRONROUT0000000009046945,Chemin,5,Non,En service,"LINESTRING (867176.9 6558772.9, 867178.6 65587..."
9997,TRONROUT0000000009046947,Route à 1 chaussée,5,Non,En service,"LINESTRING (867176.9 6558772.9, 867180.2 65587..."
9998,TRONROUT0000000009046956,Chemin,5,Non,En service,"LINESTRING (865686.2 6558785.9, 865714.9 65587..."
